## Let's figure out how to section the data and massage it into whatever form we want to input it as

### Then we can paste the streamlined version into the full training notebooks

### Load Data and do 2d analysis

In [61]:
import sys
import numpy as np
from dataframework.src.datasets.vpicdataset import VPICDataset
from numpy.random import default_rng as generator
import h5py

In [62]:
def clear_dataset(group, key):
    try:  # if the dataset exists from previous failed attempts to do this, delete it
        del group[key]
        print(f"Overwrote existing dataset {group, key}")
    except:
        pass    

In [63]:
zooms = [[-np.inf,np.inf], [-40, 40]]
time_idx = 31 #the time index we are processing rn
smoothing = 3

In [64]:
files = ['/tigress/kendrab/03082021/data.h5','tigress/kendrab/03082021/info']
writepath = '/tigress/kendrab/03082021'
unique_seed = 308
kwargs = {'get_vars' : ['bx','by','bz','jy','vz']}
sim_dset = VPICDataset(vpicfiles=files, **kwargs)


NO PARAMS ADDED, FUNCTIONALIITY NOT ADDED YET!!!! SORRY
Added bx Variable
Added by Variable
Added bz Variable
Added jy Variable
Added vz Variable


In [65]:
# zoom in
desired_time = sim_dset.timeseries[time_idx]
dt = sim_dset.timeseries[1] - sim_dset.timeseries[0]
sim_dset = sim_dset.ndslice(timelims=[desired_time - dt/2, desired_time + dt/2], zooms=zooms)

In [66]:
# do the thing
sim_dset.find_structures(b1_name='bz', b2_name='bx')

Finding structures at simulation time 309.89697265625
parameter d_per_de = 4
Added bz_smooth Variable
Added bx_smooth Variable
Added flux_fn Variable
Number of nulls:  46
parameter x_coords = [[ 114.97886056  178.42360713]
 [ 318.49994352  186.67419156]
 [ 588.28429281  165.17121359]
 [ 833.6854744   153.18075265]
 [1021.79858523  145.09320903]
 [1273.45605448  164.75710861]
 [1542.02943115  190.36864923]
 [1464.09212635  150.45679052]
 [1550.82189342  216.56050568]
 [1379.07706725  205.0737245 ]
 [4391.58249611  173.56437832]
 [5487.93240191  176.58871828]
 [7191.34066627  175.71995566]
 [7206.04018472  175.90010716]
 [7239.92367879  171.97792374]
 [7294.40415335  170.48507449]
 [7438.98319144  187.63852332]
 [7551.93398553  174.02999643]
 [7730.79265274  191.32104545]
 [8055.25046227  196.7431513 ]
 [8273.85461172  215.79643651]
 [8586.85965862  167.91299759]
 [8804.45675899  185.04524977]]
parameter o_coords = [[  46.87904234  192.20138185]
 [ 210.30322402  167.5862849 ]
 [ 442.8789

### Take random slices

In [67]:
samples = 1000
min_dist = 30
max_dist = 200
rand = generator(seed=unique_seed*time_idx)

def random_bounded(bounds):
    """ Use numpy to generate an array of random numbers with specified mins and maxes
    
    Parameters
    ----------
    bounds : array, shape (...,2)
        holds the bounds needed for each point in the random array. bounds[...,0] has mins
        
    Returns
    -------
    rands : array, shape (...)
        the array of random numbers returned
    """
    unit_rands = rand.random(size=bounds.shape[:-1])
    rands = (bounds[...,1] - bounds[...,0])*unit_rands + bounds[...,0]
    
    return rands

def mtail_transform(sim_data):
    """ The geometry of the simulation is different wrt the magnetotail. Keeping x the same, bx will be
    the mtail's 'bz' and bz will be the mtails 'bx'. For this the z and y components must be flipped 180
    THIS IS NOT A GOOD GENERALIZABLE FUNCTION!!!! REUSE WITH EXTREME CAUTION!!!
    Assumes origin centered etc.
    """
    # Flip z component of everybody
    for key in sim_data.variables.keys():
        sim_data.variables[key].data = np.flip(sim_data.variables[key].data, axis=-1)
    # Flip vector quantities
    sim_data.variables['bz'].data *= -1
    sim_data.variables['jy'].data *= -1
    sim_data.variables['vz'].data *= -1
    
    

In [68]:
bounds = sim_dset.bounds(time=False)
segment_bounds = np.expand_dims(bounds, axis=-2)
segment_bounds = np.concatenate([segment_bounds for i in range(2)], axis=-2)
varkwargs = {'separatricesinterp' : 'nearest', 'o_structuresinterp' : 'nearest',
            'current_sheetsinterp' : 'nearest'}  # to use nearest interpolation for topography info
endpts_list = []
# slices_dict format: slices_dict['variable'] = [[var_from_slice_1, var_from_slice_2, ...]]
# Also stores the parameters
slices_dict={}
# transform the data to a more magnetotail-esque config
mtail_transform(sim_dset)

for i in range(samples):
    for j in range(1000):
        endpts = np.array(random_bounded(segment_bounds))  # in [[minx, maxx], [miny, maxy]] format
        if not np.all(np.abs(endpts[:,0] - endpts[:,1]) > 1):
            continue
        if max_dist > np.linalg.norm(endpts[:,0] - endpts[:,1]) > min_dist:
            break
    else:
        raise Error("Suitable random 1d segment not found, check minimum and maximum distances.")    
    slce = sim_dset.ndslice(set_pts=endpts.T, zooms=endpts, **varkwargs)
    endpts_list.append(endpts.T)  # to make first dimension the number of points
    # sanity check variable to make sure the slices are happening correctly
    orig_mesh = np.stack([slce.params['zero_pt'] 
                          + slce.params['unit_vec'] * slce.default_mesh[0][i] 
                          for i in range(len(slce.default_mesh[0]))], axis=-1)
    slce.add_var('x0', slce.timeseries, slce.default_mesh, orig_mesh[0,:])
    slce.add_var('x1', slce.timeseries, slce.default_mesh, orig_mesh[1,:])    
    for var in slce.variables.keys(): # this will lose info if the variables have different meshes
                                       #   so will need to interpolate all to the same mesh first to avoid
        slices_dict.setdefault(var,[]).append(slce.variables[var].data.reshape(-1))
    for param in slce.params.keys(): 
        slices_dict.setdefault(param,[]).append(slce.params[param])
    cs = slce.variables['current_sheets'].data[0]
    os = slce.variables['o_structures'].data[0]
    slices_dict.setdefault('s',[]).append(slce.default_mesh[0].reshape(-1))
    # neither 0
    # os 1
    # cs 2
    # both 3
    slices_dict.setdefault('topo',[]).append((2*cs+os).reshape(-1))


overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
Added x0 Variable
Added x1 Variable
overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
Added x0 Variable
Added x1 Variable
overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
Added x0 Variable
Added x1 Variable
overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_shee

### Save the slice data to disk

In [69]:
writefile = h5py.File(writepath+"/"+str(samples)+"samples_idx"+str(time_idx)+'_' +''.join(kwargs['get_vars']) +'.hdf5', 'a')

for key in slices_dict.keys():  # this is a mess. Is this how you are supposed to use try except blocks? Prob not
    clear_dataset(writefile, key)
    
    if type(slices_dict[key]) is np.ndarray:
        dset = writefile.create_dataset(key, data=slices_dict[key])
        continue
        
    try:
        dtype = h5py.vlen_dtype(type(slices_dict[key][0].flatten()[0])) #vlen dtype is list of arrays
        print(key, type(dtype))
        dset = writefile.create_dataset(key, (len(slices_dict[key]),), dtype=dtype)
        for i in range(len(slices_dict[key])): #ew :(
            writefile[key][i] = slices_dict[key][i] 
              
    except:
        clear_dataset(writefile, key)  # clear failed attempt
        dt = h5py.special_dtype(vlen=str)
        dset = writefile.create_dataset(key, (len(slices_dict[key]),), dtype=dt)
        print(key, type(dtype))
        for i in range(len(slices_dict[key])): #ew :(
            writefile[key][i] = str(slices_dict[key][i])

Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bx')
bx <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'by')
by <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bz')
bz <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'jy')
jy <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'vz')
vz <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bz_smooth')
bz_smooth <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bx_smooth')
bx_smooth <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx31_bxbybzjyvz.hdf5" (mode r+)>, 'flux_fn')
flux_fn <class 'numpy.dtype'>
Overwrote existing

close the output file

In [70]:
writefile.close()

### Write the 2d stuff to a separate file

In [71]:
writefile = h5py.File(writepath+"/"+"idx"+str(time_idx)+'_' +''.join(kwargs['get_vars']) +'.hdf5', 'a')

clear_dataset(writefile, 'default_x') # save mesh
dset = writefile.create_dataset('default_x', data=sim_dset.default_mesh[0])
clear_dataset(writefile, 'default_z') # save mesh
dset = writefile.create_dataset('default_z', data=sim_dset.default_mesh[1])
for key in sim_dset.variables.keys():  # this is a mess. Is this how you are supposed to use try except blocks? Prob not
    clear_dataset(writefile, key)
    dset = writefile.create_dataset(key, data=sim_dset.variables[key].data)

for key in sim_dset.params.keys():
    clear_dataset(writefile, key)
    dset = writefile.create_dataset(key, data=sim_dset.params[key])

Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'default_x')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'default_z')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bx')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'by')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bz')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'jy')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'vz')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bz_smooth')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'bx_smooth')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'flux_fn')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf5" (mode r+)>, 'fluxfn_hessian_det')
Overwrote existing dataset (<HDF5 file "idx31_bxbybzjyvz.hdf

close the output file

In [72]:
writefile.close()

### Visualize the slices (optional)

In [80]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

flux_fn = sim_dset.variables['flux_fn']

X,Y = np.meshgrid(*flux_fn.mesh, indexing='ij')
fig, ax = plt.subplots(figsize=(10,2))
ctr =ax.contour(X,Y,flux_fn.data[0], levels=100)
plt.colorbar(mappable=ctr)
paths = LineCollection(endpts_list, colors='black')
ax.add_collection(paths)

for endpts in endpts_list:
    print(endpts[0])
    ax.scatter(*endpts[0], color='black') # marks starting pt
    
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[863.69870278 -19.13197912]
[-911.45377277  -24.20795462]
[811.02330302   9.81183329]
[-134.51267611    5.55148645]
[126.43913838   4.3955497 ]
[-448.92011404  -16.29089256]
[7.50064605e+02 3.34641884e-01]
[-184.76503628    5.46587066]
[-848.6831127   -24.37636023]
[-559.77324769   -8.29273678]
[486.0073411   39.53537517]
[-0.45279935 -6.86017183]
[ -0.06392445 -24.80043744]
[-41.90510441 -26.17216215]
[276.98822386  -6.86229521]
[-301.97161395  -28.52416564]
[-27.37997389 -33.09812866]
[589.44354746  18.67497793]
[-717.57157259   18.31319085]
[-300.52219622    6.34857521]
[-28.85746137 -19.91220336]
[45.04940966  1.39595705]
[-130.07742365  -28.97376093]
[-366.04795982  -12.93742125]
[-392.37675688  -27.32997918]
[617.90467724 -29.6112028 ]
[-600.23577939   26.83637751]
[-906.77903239   14.6591829 ]
[256.92702868 -22.39161009]
[463.18876451  32.59670126]
[-417.79910698    4.76931713]
[ 51.48191154 -14.3306133 ]
[-630.94320316    1.98751974]
[463.60490594  21.82333827]
[165.17310664   